# KEGG Analysis

# Phase 2 vs. Phase 2

## 0. load libraries

In [21]:
library(tidyverse)
library(ggupset) 
library(enrichplot)
library(clusterProfiler) # for GSEA() and gseKEGG()

## 1. read CSVs

In [42]:
# get list of files
files <- list.files(
    path = '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes',
    pattern = '\\.csv$',
    full.names = TRUE
    )

head(files)

[1] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bb_cc.csv"
[2] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_bb.csv"
[3] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_cb.csv"
[4] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_cc.csv"
[5] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_hc.csv"
[6] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_wc.csv"

`gseKEGG` takes in the whole list but uses LFC values (and wants them ordered, similar to GSEA - from what I've read, this is "better" than `enrichKEGG` bc of adding LFC, which allows your results to be more informative

In [43]:
names(files) <- gsub("^DEG_", "", tools::file_path_sans_ext(basename(files)))

df_list <- lapply(files, function(f) {
  df <- read.csv(f)

  # remove LOC from gene IDs
  df$X <- sub("^LOC", "", df$X)

  df
})

names(df_list) <- names(files)

all_df <- lapply(df_list, function(df) {
  geneList <- as.numeric(df$log2FoldChange)
  names(geneList) <- df$X

  # remove NA values
  geneList <- geneList[!is.na(geneList)]

  # sort decreasing
  sort(geneList, decreasing = TRUE)
})

names(all_df)
head(all_df$wc_cc)

[1] "bb_cc" "bc_bb" "bc_cb" "bc_cc" "bc_hc" "bc_wc" "bh_hh" "bw_ww" "cb_bb"
[10] "cb_bc" "cb_cc" "cb_ch" "cb_cw" "ch_cc" "ch_hc" "cw_cc" "cw_ch" "cw_wc"
[19] "hb_bb" "hb_bh" "hb_cb" "hc_cc" "hc_ch" "hc_hh" "hh_cc" "hh_ch" "wb_bb"
[28] "wb_bw" "wb_cb" "wc_cc" "wc_cw" "wc_hc" "wc_ww" "wh_hw" "ww_cc" "ww_cw"

111132404 111105680 111112633 111103543 111113858 111109183 
 45.09222  43.67937  42.75454  40.28405  38.87249  34.30553

## 2. run `gseKEGG`

### 1. effect of single exposure after period of recovery
- CC vs. WC
- CC vs. HC
- CC vs. BC

In [44]:
# CC vs. WC
k.cc.wc <- gseKEGG(geneList = all_df$wc_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.wc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.54% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [45]:
# CC vs. HC
k.cc.hc <- gseKEGG(geneList = all_df$hc_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.hc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.57% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”


,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
cvn04144,cvn04144,Endocytosis - Crassostrea virginica (eastern oyster),214,0.9957203,1.341435,0.000172953,0.02421342,0.02421342,56,"tags=1%, list=0%, signal=1%",111110117/111099630


endocytosis - cell membrane engulfs external substances, carries material inside the cell

enriched pathway in HC (I think that's how you interpret...)

enriched GO terms for this pathway are involved in cell migration/organization

In [47]:
# CC vs. BC
k.cc.bc <- gseKEGG(geneList = all_df$bc_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.bc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.55% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 2. effect of single late exposure 
- CC vs. CW
- CC vs. CH
- CC vs. CB

In [48]:
# CC vs. CW
k.cc.cw <- gseKEGG(geneList = all_df$cw_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.cw)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.77% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [49]:
# CC vs. CH
k.cc.ch <- gseKEGG(geneList = all_df$ch_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.ch)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.6% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [50]:
# CC vs. CB
k.cc.cb <- gseKEGG(geneList = all_df$cb_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.cb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.75% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 3. Effect of timing of initial stress exposure
- CW vs. WC
- CH vs. HC
- CB vs. BC

In [51]:
# CW vs. WC
k.cw.wc <- gseKEGG(geneList = all_df$cw_wc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cw.wc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.62% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [52]:
# CH vs. HC
k.ch.hc <- gseKEGG(geneList = all_df$ch_hc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.ch.hc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.41% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [53]:
# CB vs. BC
k.cb.bc <- gseKEGG(geneList = all_df$cb_bc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cb.bc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.51% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways